In [1]:
import requests
import pandas as pd
import xlwings as xw

def clean_vin(vin):
    """Cleans up the VIN by trimming spaces and replacing O->0, I->1."""
    return vin.strip().upper().replace("O", "0").replace("I", "1")

def decode_vins_batch(vin_list, original_vins):
    """Calls the NHTSA vPIC Batch API to decode multiple VINs."""
    url = "https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/"
    vin_data = ";".join(vin_list)
    payload = {"format": "json", "data": vin_data}
    
    try:
        response = requests.post(url, data=payload, verify=False, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if "Results" not in data or not data["Results"]:
            print("Warning: No valid results returned from API.")
            return []
        
        decoded_vins = []
        for original_vin, result in zip(original_vins, data["Results"]):
            decoded_vins.append([
                original_vin,  # Original untouched VIN
                clean_vin(original_vin),  # Cleaned VIN
                result.get("Make", "N/A"),
                result.get("Model", "N/A"),
                result.get("VehicleType", "N/A"),
                result.get("GVWR", "N/A")
            ])
        return decoded_vins
    except requests.exceptions.RequestException as e:
        print(f"Error making batch request: {e}")
        return []

def process_vins_from_excel(file_path):
    """Reads VINs from the given Excel file, decodes them, and writes the results back safely using xlwings."""
    sheet_input = "New Intake Form"
    sheet_output = "Vehicle List"

    # Open Excel workbook
    app = xw.App(visible=False)
    wb = xw.Book(file_path)
    
    try:
        # Read VINs from Excel
        ws_input = wb.sheets[sheet_input]
        df_input = pd.read_excel(file_path, sheet_name=sheet_input)

        original_vins = df_input['VIN'].astype(str).tolist()
        cleaned_vins = [clean_vin(vin) for vin in original_vins]

        # Decode VINs in batches
        batch_size = 50
        decoded_results = []
        for i in range(0, len(cleaned_vins), batch_size):
            batch = cleaned_vins[i:i + batch_size]          
            orig_batch = original_vins[i:i + batch_size]  # Keep track of original VINs
            decoded_results.extend(decode_vins_batch(batch, orig_batch))



        # Write to output sheet
        ws_output = wb.sheets[sheet_output] if sheet_output in [s.name for s in wb.sheets] else wb.sheets.add(sheet_output)
        # ws_output.clear_contents()

        # Write headers and data
        headers = ["Original VIN", "Cleaned VIN", "Make", "Model", "Vehicle Type", "GVWR"]
        ws_output.range("A1").value = headers
        ws_output.range("A2").value = decoded_results
        
        # Save workbook
        wb.save()
        print("VIN decoding process completed successfully.")
    finally:
        wb.close()
        app.quit()

file_path = r"C:\Users\ez4ke.KDAWG\Desktop\astrus\Auto Schedule XML Import Export.xlsm"
process_vins_from_excel(file_path)


KeyError: 'VIN'

In [ ]:
sheet_input = "New Intake Form"
sheet_output = "Vehicle List"

# Open Excel workbook
app = xw.App(visible=False)
wb = xw.Book(file_path)

 
# Read VINs from Excel
ws_input = wb.sheets[sheet_input]
df_input = pd.read_excel(file_path, sheet_name=sheet_input)

original_vins = df_input['VIN'].astype(str).tolist()
cleaned_vins = [clean_vin(vin) for vin in original_vins]

# Decode VINs in batches
batch_size = 50
decoded_results = []
for i in range(0, len(cleaned_vins), batch_size):
    batch = cleaned_vins[i:i + batch_size]          
    orig_batch = original_vins[i:i + batch_size]  # Keep track of original VINs
    decoded_results.extend(decode_vins_batch(batch, orig_batch))



# Write to output sheet
ws_output = wb.sheets[sheet_output] if sheet_output in [s.name for s in wb.sheets] else wb.sheets.add(sheet_output)
# ws_output.clear_contents()

# Write headers and data
headers = ["Original VIN", "Cleaned VIN", "Make", "Model", "Vehicle Type", "GVWR"]
ws_output.range("A1").value = headers
ws_output.range("A2").value = decoded_results

# Save workbook
wb.save()
print("VIN decoding process completed successfully.")

wb.close()
app.quit()

c:\Users\kzhang2\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpic.nhtsa.dot.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


VIN decoding process completed successfully.


In [17]:
wb.close()
app.quit()